In [188]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

url='https://livingwage.mit.edu/counties/'

start=36001
end=36123
codes=[]
for i in range(start, end+1, 2):
    codes.append(str(i))

r=requests.get('https://livingwage.mit.edu/counties/36001')
html_doc=r.text
soup=BeautifulSoup(html_doc)

expenseTable=soup.find_all(class_='results_table table-striped expense_table')[0]
col_names=[]

for col_name in expenseTable.find_all(class_='text'):
    col_names.append(col_name.text)
print(col_names)


['Food', 'Child Care', 'Medical', 'Housing', 'Transportation', 'Civic', 'Internet & Mobile', 'Other', 'Required annual income after taxes', 'Annual taxes', 'Required annual income before taxes']


In [189]:
requestPerCounty=[]
for code in codes:
    requestPerCounty.append(BeautifulSoup(requests.get('https://livingwage.mit.edu/counties/'+code).text))

In [200]:
tablePerCounty=[]
for request in requestPerCounty:
    tablePerCounty.append(request.find_all(class_='results_table table-striped expense_table')[0])

dollarAmount=[]
for table in tablePerCounty:
    total=[]
    for  trow in table.find_all('tr')[2:]:
        total.append(trow.find_all('td')[1].text.strip())
    dollarAmount.append(total)

In [191]:
nys_counties = ["Albany", "Allegany", "Bronx", "Broome", "Cattaraugus", "Cayuga", "Chautauqua", "Chemung", "Chenango", "Clinton", "Columbia", "Cortland", 
                "Delaware", "Dutchess", "Erie", "Essex", "Franklin", "Fulton", "Genesee", "Greene", "Hamilton", "Herkimer", "Jefferson", "Kings", "Lewis", 
                "Livingston", "Madison", "Monroe", "Montgomery", "Nassau", "New York", "Niagara", "Oneida", "Onondaga", "Ontario", "Orange", "Orleans", 
                "Oswego", "Otsego", "Putnam", "Queens", "Rensselaer", "Richmond", "Rockland", "St. Lawrence", "Saratoga", "Schenectady", "Schoharie", 
                "Schuyler", "Seneca", "Steuben", "Suffolk", "Sullivan", "Tioga", "Tompkins", "Ulster", "Warren", "Washington", "Wayne", "Westchester", 
                "Wyoming", "Yates"]


In [192]:
data=np.array(dollarAmount)
df=pd.DataFrame(data, columns=col_names)
df['County']=nys_counties
df.set_index('County', inplace=True)
pd.set_option('display.max_rows', None)  

df.to_csv('countyData.csv')
#df

Now doing the same for New York State as a whole from 2015-2025

In [193]:
years=['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']

links=['https://web.archive.org/web/20151220151322/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20161207171550/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20171207053953/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20181026231852/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20190811183231/http://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20201124071353/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20211016060013/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20221223191058/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20231202131633/https://livingwage.mit.edu/states/36',
       'https://web.archive.org/web/20241230163723/https://livingwage.mit.edu/states/36',
       'https://livingwage.mit.edu/states/36'
       ]

In [194]:
requestPerYear=[]
for link in links:
    requestPerYear.append(BeautifulSoup(requests.get(link).text))

In [195]:
tablePerYear=[]
for request in requestPerYear:
    try:
        tablePerYear.append(request.find_all(class_='table table-striped table-condensed expenses_table')[0])
    except:
        tablePerYear.append(request.find_all(class_='results_table table-striped expense_table')[0])

count=0
dollarAmountperYear=[]
for table in tablePerYear:
    total=[]
    if count>4:
        for  trow in table.find_all('tr')[2:]:
            total.append(trow.findAll('td')[1].text.strip())
    else:
        for  trow in table.find_all('tr')[1:]:
            total.append(trow.findAll('td')[1].text.strip())
    count+=1
    dollarAmountperYear.append(total)

C:\Users\fatou\AppData\Local\Temp\ipykernel_50248\557875152.py:17: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  total.append(trow.findAll('td')[1].text.strip())
C:\Users\fatou\AppData\Local\Temp\ipykernel_50248\557875152.py:14: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  total.append(trow.findAll('td')[1].text.strip())


In [196]:
data3=np.array(dollarAmountperYear[9:11])
df2=pd.DataFrame(data3, columns=col_names)

col_names2=col_names.copy()
col_names2.remove('Internet & Mobile')
data4=np.array(dollarAmountperYear[6:9])
df3=pd.DataFrame(data4, columns=col_names2)

col_names3=col_names2.copy()
col_names3.remove('Civic')
data4=np.array(dollarAmountperYear[:6])
df4=pd.DataFrame(data4, columns=col_names3)


allYears = pd.concat([df2, df3, df4], axis=0)
dollarAmountperYear
allYears['year']=years
allYears.set_index('year', inplace=True)
allYears.to_csv('nysData.csv')